In [5]:
from functools import wraps
from typing import Callable, Generator

# 데코레이터 함수: 함수의 실행 시간을 측정하는 기능을 추가함
def timeit(func: Callable) -> Callable:
    import time
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"실행 시간: {end_time - start_time: .4f}초")
        return result
    
    return wrapper

# 제너레이터 함수: 메모리를 아끼기 위해 한 번에 하나의 값을 생성
def fibonacci(n: int) -> Generator[int, None, None]:
    # n번째 피보나치 수열까지 생성
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a+b

# 고차 함수: 다른 함수를 인자로 받아 처리
def apply_operation(data: list[int], operation: Callable[[int], int]) -> list[int]:
    # 주어진 함수(operation)를 데이터 리스트에 적용
    return [operation(x) for x in data]

# 함수형 프로그래밍: 람다 함수 사용
square = lambda x: x ** 2
cube = lambda x: x ** 3

# 주요 실행 코드
@timeit	# 데코레이터 적용: 실행 시간을 측정
def main():
    fib_list = list(fibonacci(10))	# 10번째 피보나치 수열까지 생성
    print("피보나치 수열:", fib_list)
    
    # 제곱 함수 적용
    squared_list = apply_operation(fib_list, square)
    print("제곱된 값:", squared_list)
    
    # 세제곱 함수 적용
    cubed_list = apply_operation(fib_list, cube)
    print("세제곱된 값:", cubed_list)

# main 함수 실행
if __name__ == "__main__":
    main()

피보나치 수열: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
제곱된 값: [0, 1, 1, 4, 9, 25, 64, 169, 441, 1156]
세제곱된 값: [0, 1, 1, 8, 27, 125, 512, 2197, 9261, 39304]
실행 시간:  0.0010초


In [3]:
class SingletonMeta(type):
    # 싱글톤 패턴 구현을 위한 메타클래스
    # 싱글톤은 하나의 클래스에 대해 오직 하나의 인스턴스만 존재하도록 보장하는 디자인 패턴입니다.
    _instances = {}  # 클래스별로 생성된 인스턴스를 저장할 딕셔너리
    
    # 클래스 호출할 때 실행되는 메소드
    def __call__(cls, *args, **kwargs):
        # cls가 _instances에 없으면 새로운 인스턴스를 생성하고 저장
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        # 이미 인스턴스가 존재하면 그 인스턴스를 반환
        return cls._instances[cls]

# 싱글톤 클래스: 동일한 인스턴스만 생성
class SingletonClass(metaclass=SingletonMeta):
    def __init__(self, value):
        self.value = value  # 인스턴스에 저장할 값

# 컨텍스트 관리자: 특정 자원의 사용을 관리
class FileManager:
    # 파일을 안전하게 열고 닫는 컨텍스트 관리자
    def __init__(self, filename, mode):
        self.filename = filename  # 열 파일의 이름
        self.mode = mode          # 파일 열기 모드 (예: 읽기, 쓰기)
        self.file = None          # 나중에 파일 객체를 저장할 변수
    
    def __enter__(self):
        # 컨텍스트가 시작되면 파일을 엽니다.
        print(f"{self.filename} 파일을 엽니다.")
        self.file = open(self.filename, self.mode)
        return self.file  # 파일 객체 반환 (with 구문에서 사용)
    
    def __exit__(self, exc_type, exc_value, traceback):
        # 컨텍스트가 끝나면 파일을 닫습니다.
        print(f"{self.filename} 파일을 닫습니다.")
        if self.file:
            self.file.close()  # 파일이 열려있다면 닫기

# 데코레이터 체이닝: 여러 데코레이터를 체인 형태로 적용
def bold(func):
    # HTML 태그를 사용해 텍스트를 굵게 만듦
    def wrapper(*args, **kwargs):
        return f"<b>{func(*args, **kwargs)}</b>"
    return wrapper

def italic(func):
    # HTML 태그를 사용해 텍스트를 기울임
    def wrapper(*args, **kwargs):
        return f"<i>{func(*args, **kwargs)}</i>"
    return wrapper

@bold	# 먼저 bold 데코레이터가 적용
@italic	# 마지막으로 italic 데코레이터가 적용
def get_text():
    return "Python 고급 프로그래밍"  # 실제 함수는 이 텍스트를 반환

# 주요 실행 코드
if __name__ == "__main__":
    # 싱글톤 클래스 사용
    s1 = SingletonClass("첫 번째 인스턴스")
    s2 = SingletonClass("두 번째 인스턴스")
    
    # s1과 s2가 같은 객체인지 확인 (싱글톤이므로 같은 객체여야 함)
    print(f"s1과 s2는 같은 객체인가? {s1 is s2}")
    # s1과 s2가 동일한 값을 가지는지 확인
    print(f"s1의 값: {s1.value}")
    print(f"s2의 값: {s2.value}")
    
    # 컨텍스트 관리자 사용
    with FileManager("sample.txt", "w") as file:
        # 파일을 열고 텍스트를 작성
        file.write("파일에 텍스트를 씁니다.")
    
    # 데코레이터 체이닝 예시
    # get_text 함수는 italic -> bold 순으로 데코레이터가 적용됨
    print(get_text())  # <b><i>Python 고급 프로그래밍</i></b> 출력


s1과 s2는 같은 객체인가? True
s1의 값: 첫 번째 인스턴스
s2의 값: 첫 번째 인스턴스
sample.txt 파일을 엽니다.
sample.txt 파일을 닫습니다.
<b><i>Python 고급 프로그래밍</i></b>
